# BM25
BM25是一种排名函数，用于信息检索领域，以在给定查询下对文档集合进行排序。  

BM25算法的核心思想是根据词项的出现频率来评估文档对于查询的相关性，  
同时通过引入两个调节参数（k1和b）来控制词项频率（TF）的饱和度和文档长度的影响。

具体来说，BM25的评分函数如下：  

$\text{Score}(D, Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})}$

其中：
- $D$ 是文档，
- $Q$ 是查询，
- $q_i$ 是查询中的第 $i$ 个词项，
- $f(q_i, D)$ 是词项 $q_i$ 在文档 $D$ 中的出现频率，
- $|D|$ 是文档 $D$ 的长度，
- $\text{avgdl}$ 是集合中所有文档长度的平均值，
- $k_1$ 和 $b$ 是可调节的参数，通常 $k_1$ 在1.2到2.0之间，$b$ 等于0.75，
- $\text{IDF}(q_i)$ 是逆文档频率，用于衡量词项的普遍重要性。

通过这种方式，BM25算法能够有效地衡量文档和查询之间的相关性，广泛应用于全文搜索引擎和信息检索系统中。

In [2]:
import math
from collections import Counter

def compute_idf(doc_list):
    '''计算一组文档中每个词项的逆文档频率
    doc_list : 文档列表
    '''
    # 存储每个词项的IDF得分
    idf_scores = {}
    # 文档列表中文档的总数
    total_docs = len(doc_list)
    # 计数每个词项出现的文档数（即文档频率）
    doc_freq = Counter([word for doc in doc_list for word in set(doc)])
    # 遍历doc_freq字典的每个项，word是词项，df是该词项出现的文档数。
    for word, df in doc_freq.items():
        # 根据公式计算每个词项的IDF得分
        idf_scores[word] = math.log((total_docs - df + 0.5) / (df + 0.5) + 1)
    # 返回包含所有词项及其IDF得分的字典
    return idf_scores

In [3]:
def bm25_score(doc, query, idf_scores, avgdl, k1=1.5, b=0.75):
    '''计算给定文档相对于一个查询的BM25得分
    doc : 当前需要计算得分的文档
    query : 用户的查询
    idf_scores : 一个字典，包含每个词项的IDF得分。
    avgdl : 文档集合中文档的平均长度
    k1 和 b : BM25算法中的两个调节参数，分别有默认值1.5和0.75。
    '''
    score = 0.0
    # 计算当前文档的长度，即文档中词项的数量。
    doc_len = len(doc)
    # 使用Counter统计当前文档中每个词项出现的次数。
    doc_freqs = Counter(doc)
    # 遍历查询中的每个词项
    for word in query:
        # 检查当前词项是否在idf_scores字典中，即是否计算过IDF得分。
        if word in idf_scores:
            # 获取当前词项在文档中的频率
            df = doc_freqs[word]
            # 获取当前词项的IDF得分
            idf = idf_scores[word]
            # 根据BM25的计算公式，计算每个词项对总得分的贡献，更新当前文档的得分。
            score += idf * (df * (k1 + 1)) / (df + k1 * (1 - b + b * (doc_len / avgdl)))
    # 返回计算得到的文档BM25得分
    return score

In [4]:
def compute_avgdl(doc_list):
    '''用于计算给定文档集合中文档的平均长度'''
    # 计算了所有文档的总长度
    total_length = sum(len(doc) for doc in doc_list)
    # 计算文档集合的平均文档长度
    avgdl = total_length / len(doc_list)
    # 返回计算出的平均文档长度
    return avgdl

In [5]:
# 举例演示
doc_list = [["小猫", "在", "屋顶", "上"],
               ["小狗", "和", "小猫", "是", "好朋友"],
               ["我", "喜欢", "看", "书"]]
         
query = ["小猫", "在哪里"]

# 计算idf_scores和avgdl（重新使用之前定义的函数）
idf_scores = compute_idf(doc_list)
avgdl = compute_avgdl(doc_list)

# 对每个文档计算BM25得分
scores = []
for doc in doc_list:
    score = bm25_score(doc, query, idf_scores, avgdl)
    scores.append(score)

print(scores)

[0.4868563490194871, 0.4395717395823426, 0.0]
